In [3]:
import pandas as pd
import numpy as np

In [4]:
# Load data
path    = "MIMIC-IV v2.2\icu\chartevents.csv"
reader  = pd.read_csv(path, chunksize=10000000)

In [5]:
# Calculates AKI stage based on KDIGO criteria: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4412317/pdf/rbccv-29-03-0299.pdf
def calculate_aki_stage(group):
    group['aki_stage'] = 0
    baseline_creatinine = group.iloc[0]['baseline_creatinine']
    for i in range(len(group)):
        current_creatinine = group.iloc[i]['valuenum']
        time_diff = (group.iloc[i]['charttime'] - group.iloc[0]['charttime']).total_seconds() / 3600  # in hours
        
        # Baseline creatinine
        if current_creatinine >= 4.0 or current_creatinine >= 3.0 * baseline_creatinine:
            group.at[group.index[i], 'aki_stage'] = 3
        elif current_creatinine >= 2.0 * baseline_creatinine:
            group.at[group.index[i], 'aki_stage'] = 2
        elif current_creatinine >= 1.5 * baseline_creatinine:
            group.at[group.index[i], 'aki_stage'] = 1
        
        # Last 48 hours
        if time_diff <= 48:
            if current_creatinine - baseline_creatinine >= 0.3:
                group.at[group.index[i], 'aki_stage'] = max(group.at[group.index[i], 'aki_stage'], 1)
    return group

In [6]:
for i, df in enumerate(reader):
    # Get only creatinine values and convert charttime to datetime
    creatinine_df = df[df['itemid'] == 220615]
    creatinine_df = creatinine_df[['subject_id', 'stay_id', 'charttime', 'valuenum', 'warning']]
    creatinine_df['charttime'] = pd.to_datetime(creatinine_df['charttime'])

    # Set baseline
    baseline_df = creatinine_df.groupby('subject_id')['valuenum'].quantile(0.25).reset_index()
    # baseline_df = creatinine_df.groupby('subject_id')['valuenum'].min().reset_index()
    baseline_df.columns = ['subject_id', 'baseline_creatinine']
    creatinine_df = creatinine_df.merge(baseline_df, on='subject_id')

    # Calculate AKI stage
    aki_patients = creatinine_df.groupby('subject_id').apply(calculate_aki_stage, include_groups=True).reset_index(drop=True)

    # Save to file without overwriting
    path = 'aki_patients_low_quartile.csv'
    if i == 0:
        aki_patients.to_csv(path, index=False)
    else:
        aki_patients.to_csv(path, mode='a', header=False, index=False)

    print(f"{i}")

C:\Users\Andrew Chen\AppData\Local\Temp\ipykernel_21212\3009373029.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aki_patients = creatinine_df.groupby('subject_id').apply(calculate_aki_stage, include_groups=True).reset_index(drop=True)


0


C:\Users\Andrew Chen\AppData\Local\Temp\ipykernel_21212\3009373029.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aki_patients = creatinine_df.groupby('subject_id').apply(calculate_aki_stage, include_groups=True).reset_index(drop=True)


1


C:\Users\Andrew Chen\AppData\Local\Temp\ipykernel_21212\3009373029.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aki_patients = creatinine_df.groupby('subject_id').apply(calculate_aki_stage, include_groups=True).reset_index(drop=True)


...

C:\Users\Andrew Chen\AppData\Local\Temp\ipykernel_21212\3009373029.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aki_patients = creatinine_df.groupby('subject_id').apply(calculate_aki_stage, include_groups=True).reset_index(drop=True)


29


C:\Users\Andrew Chen\AppData\Local\Temp\ipykernel_21212\3009373029.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aki_patients = creatinine_df.groupby('subject_id').apply(calculate_aki_stage, include_groups=True).reset_index(drop=True)


30
31


C:\Users\Andrew Chen\AppData\Local\Temp\ipykernel_21212\3009373029.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aki_patients = creatinine_df.groupby('subject_id').apply(calculate_aki_stage, include_groups=True).reset_index(drop=True)
